<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Directories" data-toc-modified-id="Directories-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Directories</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

In [1]:
import scipy.stats as stats
import pandas as pd
import numpy as np
from scipy.integrate import quad
import plotly.plotly  as py
import plotly.graph_objs as go

## Directories

In [2]:
main_dir = "/Volumes/LaCie/Research/NAC/"
data_dir = main_dir+"data/"
out_dir = main_dir+"output/"

## Summary

Define, $v\equiv \frac{2\pi R_*}{P_{rot}}$ and $u\equiv v \sin(I_*)$

$$p(D| cos(I_*)) = \int_0^\infty \mathcal{N}(V, \sigma_V ; v) \mathcal{N}(U, \sigma_U;v\sqrt{1-\cos^2(I_*)}) dv$$

where $\mathcal{N}$ denotes a Gaussian function of $x$ with a mean $\mu$ and standard deviation $\sigma$, and $(V, \sigma_V)$ and $(U,\sigma_U)$ are the measurements of $v$ and $u$, respectively.

In [3]:
df = pd.read_csv("{}MN_17_3616_point.csv".format(data_dir),
                index_col=[0])
df_trim = df[df.V.notnull()].reset_index(drop=True)
df_trim = df_trim[np.isfinite(df_trim.EVSINI)].reset_index(drop=True)

In [4]:
from multiprocessing import Pool, Manager, active_children

## list of columns to retrive
features = ["V", "V_sigma", "VSINI", "EVSINI"]

def integrand(u, p1, p2, cosi):
    r'''
    Parameters:
    -----------
    u : For use in scipy.integrate.quad
    p1 : scipy.stats._distn_infrastructure.rv_frozen
        Frozen distribution of "v"
    p2 : scipy.stats._distn_infrastructure.rv_frozen
        Frozen distribution of "vsini"
    sini : float
        sin(I_*) measurement for a star
    
    Returns:
    --------
    u * (p1 evaluated at u) * (p2 evaluated at u*sini)
    '''
    return u*p1.pdf(u)*p2.pdf(u*np.sqrt(1- cosi**2))

def compute_L(args, start=0.0, end=300.0):
    global integrand
    cosi, p1, p2 = args
    return quad(integrand, start, end, args=(p1,p2, cosi))[0]

pool = Pool(8)

def likelihood(cosis, v, v_sigma, vsini, vsini_sigma):
    r'''
    Computes integrand from 0 to +infinity
    
    Parameters:
    -----------
    sinis : array or list
        sinis to compute the likelihood for
    v : float
        velocity for a star
    v_sigma : float
        Error on the velocity
    vsini : float
        vsini for a star
    vsini_sigma : float
        error on vsini

    Returns:
    --------
    (pdf/norm) : ndarray
        Normalized likelihoods given a range of sini
    '''
    
    p1 = stats.norm(loc=v, scale=v_sigma)
    p2 = stats.norm(loc=vsini, scale=vsini_sigma)
    
    pdf = pool.map(compute_L, [(i, p1, p2) for i in cosis])
    #pdf = np.array([quad(integrand, 0, np.inf, args=(p1,p2, sini)) for sini in sinis])[:,0]
    
    norm = np.trapz(pdf, cosis, dx=0.01)
    
    return pdf/norm

In [5]:
Ls = Manager().list()

def main(index):
    global Ls
    global traces
    global df_trim
    global cosi
    
    if index % 10 == 0:
        print("Inspecting Star {}...".format(index))
    
    v, v_sigma, vsini, vsini_sigma = df_trim.loc[index][features].values
    ##v, v_sigma, vsini, vsini_sigma = 10, 0.5, 10, 0.5
    
    L = likelihood(cosi, v, v_sigma, vsini, vsini_sigma)
    
    Ls.append(L)

In [6]:
from joblib import Parallel, delayed
cosi = np.arange(0.0, 1.0, 0.002)

if __name__ == "__main__":
    Parallel(n_jobs=3, backend="threading")(
        map(delayed(main), range(len(df_trim))))

Inspecting Star 0...
Inspecting Star 10...
Inspecting Star 20...
Inspecting Star 30...
Inspecting Star 40...
Inspecting Star 50...


In [7]:
traces = []

for L in range(len(Ls)):
    traces.append(go.Scatter(
            x = cosi,
            y = Ls[L],
            name = df_trim.loc[L].Name
        )
    )

In [8]:
layout = go.Layout(
            xaxis={
                "title" : "$\cos(I_*)$",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1,
                "range" : [0, 1]
                },
            yaxis={
                "title" : "$L$",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1
                },
        )
figure = go.Figure(data=traces, layout=layout)
py.iplot(figure, filename="pcosi")

In [9]:
temp = pd.DataFrame(list(Ls), columns = cosi)
temp["Name"] = df_trim.Name
temp.to_csv(out_dir+"cosi.csv")